In [11]:
import sys
import os
sys.path.insert(1, 'models')

from models import position_roles_model, positions_model, user_roles_model, ous_model

positions = positions_model.get_all_positions(6000)
for pos in positions:
    user_role_id = position_roles_model.get_position_role_from_position(pos.id)
    for id in user_role_id:
        user_role = user_roles_model.get_group_from_id(id.user_role_id)
        print(user_role.name)

Jira Servicedesk_Administrator
Lederrolle
Løn læse adgang
Limited access to roles
Naturbeskyttelse agent
Naturbeskyttelse administrator
SAPA adgang
fzdfsfsdf
Tangora Administrator
Send mail ved tildeling
SD-Feriegodkender
SD-Fraværsgodkender
SD-Leder
Lederrolle
Printer 1sal 
Printer 1sal 
Adgang
Lederrolle
Adgang
Lederrolle
Adgang
ScanJour_Administrator
Printer 1sal 
Jira Servicedesk_Administrator
Lederrolle
